In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, r2_score
import joblib # Import joblib at the top with other imports
import os     # Import os for path manipulation

# --- 1. Load your Data ---
path = r'D:\Autonomous Systems A\Truck-Platooning-Simulation-CARLA\Computer vision model\line_detection_features.csv'
data = pd.read_csv(path)
df = pd.DataFrame(data)

# --- 2. Define Features (X) and Continuous Target (y) ---
continuous_target_column = 'cx'

feature_columns = [col for col in df.columns if col not in ['filename', 'line_label', continuous_target_column]]

X = df[feature_columns]
y = df[continuous_target_column]

# --- 3. Split data into training and testing sets ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 4. Scale Input Features (X) ---
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# --- 5. Scale Target Variable (y) to the range [-1, 1] ---
scaler_y = MinMaxScaler(feature_range=(-1, 1))
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

# --- 6. Train the SVR model ---
# Note: You changed kernel to 'linear'. Keep 'rbf' if it gave better previous results.
svr_model = SVR(kernel='linear', C=10.0, epsilon=0.05, gamma='scale')
svr_model.fit(X_train_scaled, y_train_scaled)

# --- 7. Make predictions ---
y_pred_scaled = svr_model.predict(X_test_scaled)

# --- Evaluate the model ---
print(f"Mean Absolute Error (scaled): {mean_absolute_error(y_test_scaled, y_pred_scaled):.4f}")
print(f"R-squared (scaled): {r2_score(y_test_scaled, y_pred_scaled):.4f}")

---

### Saving the Model and Scalers

# Create a directory to save models and scalers if it doesn't exist
save_dir = 'trained_models'
os.makedirs(save_dir, exist_ok=True)

# Define file paths for the model and scalers
model_path = os.path.join(save_dir, 'svr_line_position_model.joblib')
scaler_X_path = os.path.join(save_dir, 'scaler_X.joblib')
scaler_y_path = os.path.join(save_dir, 'scaler_y.joblib')

# Save the SVR model
joblib.dump(svr_model, model_path)
print(f"SVR Model saved successfully to '{model_path}'")

# Save the StandardScaler for features
joblib.dump(scaler_X, scaler_X_path)
print(f"Features StandardScaler saved successfully to '{scaler_X_path}'")

# Save the MinMaxScaler for the target variable
joblib.dump(scaler_y, scaler_y_path)
print(f"Target MinMaxScaler saved successfully to '{scaler_y_path}'")

print("\nAll components (model and scalers) necessary for future predictions have been saved.")

# --- How to Load Them Back for New Predictions ---
# In a new script or session, you would load them like this:
# loaded_svr_model = joblib.load(model_path)
# loaded_scaler_X = joblib.load(scaler_X_path)
# loaded_scaler_y = joblib.load(scaler_y_path)

# Then, to make a new prediction:
# new_raw_features = pd.DataFrame([[...]], columns=feature_columns) # Replace with your actual new data
# new_features_scaled = loaded_scaler_X.transform(new_raw_features)
# predicted_position_scaled = loaded_svr_model.predict(new_features_scaled)
# predicted_position_original_scale = loaded_scaler_y.inverse_transform(predicted_position_scaled.reshape(-1, 1)).flatten()

SyntaxError: unterminated string literal (detected at line 51) (484111990.py, line 51)